In [1]:
import pickle
import pandas as pd
from tqdm import tqdm

In [3]:
df = pd.read_csv("data-dir/train_data.csv")
df = df.drop(df.loc[(df.duplicated(subset=["Question"], keep=False)) & (df["Answer_possible"] == False)].index, axis=0).reset_index(drop=True)
df

,Unnamed: 0,Theme,Paragraph,Question,Answer_possible,Answer_text,Answer_start
0,2,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,True,['2003'],[526]
1,6,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What album made her a worldwide known artist?,True,['Dangerously in Love'],[505]
2,7,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Who managed the Destiny's Child group?,True,['Mathew Knowles'],[360]
3,8,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyoncé rise to fame?,True,['late 1990s'],[276]
4,9,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What role did Beyoncé have in Destiny's Child?,True,['lead singer'],[290]
...,...,...,...,...,...,...,...
75030,130309,Matter,These quarks and leptons interact through four...,How many quarks and leptons are there?,False,[],[]
75031,130310,Matter,These quarks and leptons interact through four...,What model satisfactorily explains gravity?,False,[],[]
75032,130312,Matter,These quarks and leptons interact through four...,Mass and energy can always be compared to what?,False,[],[]
75033,130314,Matter,"The term ""matter"" is used throughout physics i...",Physics has broadly agreed on the definition o...,False,[],[]


In [5]:
ques2idx = {ques: str(idx) for idx, ques in enumerate(df["Question"].unique())}
idx2ques = {value: key for key, value in ques2idx.items()}

para2idx = {para: str(idx) for idx, para in enumerate(df["Paragraph"].unique())}
idx2para = {value: key for key, value in para2idx.items()}

theme2idx = {theme: str(idx) for idx, theme in enumerate(df["Theme"].unique())}
idx2theme = {value: key for key, value in theme2idx.items()}

paragraphs = df['Paragraph']
para_idx_2_theme_idx = {}
for idx, row in df.iterrows():
    para_idx_2_theme_idx[para2idx[row['Paragraph']]
                            ] = theme2idx[row['Theme']]

for i in tqdm(range(len(df))):
    df.loc[i, "question_id"] = ques2idx[df.iloc[i]["Question"]]
    df.loc[i, "paragraph_id"] = para2idx[df.iloc[i]["Paragraph"]]
    df.loc[i, "theme_id"] = theme2idx[df.iloc[i]["Theme"]]

100%|██████████| 75035/75035 [00:27<00:00, 2719.39it/s]


In [7]:
df

,Unnamed: 0,Theme,Paragraph,Question,Answer_possible,Answer_text,Answer_start,question_id,paragraph_id,theme_id
0,2,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,True,['2003'],[526],0,0,0
1,6,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What album made her a worldwide known artist?,True,['Dangerously in Love'],[505],1,0,0
2,7,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Who managed the Destiny's Child group?,True,['Mathew Knowles'],[360],2,0,0
3,8,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyoncé rise to fame?,True,['late 1990s'],[276],3,0,0
4,9,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What role did Beyoncé have in Destiny's Child?,True,['lead singer'],[290],4,0,0
...,...,...,...,...,...,...,...,...,...,...
75030,130309,Matter,These quarks and leptons interact through four...,How many quarks and leptons are there?,False,[],[],75014,15553,360
75031,130310,Matter,These quarks and leptons interact through four...,What model satisfactorily explains gravity?,False,[],[],75015,15553,360
75032,130312,Matter,These quarks and leptons interact through four...,Mass and energy can always be compared to what?,False,[],[],75016,15553,360
75033,130314,Matter,"The term ""matter"" is used throughout physics i...",Physics has broadly agreed on the definition o...,False,[],[],75017,15554,360


In [22]:
def dummy_retriever(question, df_t, k=3):
    # return paragraph_id list
    return df_t.sample(n=k)["paragraph_id"].tolist()

In [25]:
df_test_prepared = pd.DataFrame()
themes = df["Theme"].unique()
gb_theme = df.groupby("Theme")
for t in themes:
    df_t = gb_theme.get_group(t)
    for ques in df_t["Question"].unique():
        top_k_para_idx = dummy_retriever(ques, df_t, k=3)
        print(df_t.loc[df_t["paragraph_id"].isin(top_k_para_idx)])
        
        break
    break

     Unnamed: 0    Theme                                          Paragraph  \
0             2  Beyoncé  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
1             6  Beyoncé  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
2             7  Beyoncé  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
3             8  Beyoncé  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
4             9  Beyoncé  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
5            11  Beyoncé  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
6            12  Beyoncé  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
7            14  Beyoncé  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
311         454  Beyoncé  Beyoncé names Michael Jackson as her major mus...   
312         455  Beyoncé  Beyoncé names Michael Jackson as her major mus...   
313         456  Beyoncé  Beyoncé names Michael Jackson as her major mus...   
314         457  Beyoncé  Beyoncé names Michael Jack

In [28]:
df["Theme"].dtype == str

False

In [15]:
data_t = df_t.to_dict(orient="index")

In [16]:
data_t.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

In [17]:
data_t[0]

{'Unnamed: 0': 2,
 'Theme': 'Beyoncé',
 'Paragraph': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'Question': "When did Beyonce leave Destiny's Child and become a solo singer?",
 'Answer_possible': True,
 'Answer_text': "['2003']",
 'Answer_start': '[526]',
 'question_id': '0',
 'paragraph_id': '0',
 'theme_id': '0'}

In [3]:
with open("data-dir/con_idx_2_title_idx.pkl", "rb") as f:
    con_idx_2_title_idx = pickle.load(f)

In [9]:
[key for key, val in con_idx_2_title_idx.items() if val == 11]

[897,
 898,
 899,
 900,
 901,
 902,
 903,
 904,
 905,
 906,
 907,
 908,
 909,
 910,
 911,
 912,
 913,
 914,
 915,
 916,
 917,
 918,
 919,
 920,
 921,
 922,
 923,
 924,
 925,
 926,
 927,
 928,
 929,
 930,
 931,
 932,
 933,
 934,
 935,
 936,
 937,
 938,
 939,
 940,
 941,
 942,
 943,
 944,
 945,
 946,
 947,
 948,
 949,
 950,
 951,
 952,
 953,
 954,
 955,
 956,
 957,
 958,
 959,
 960,
 961,
 962,
 963,
 964,
 965,
 966,
 967,
 968,
 969,
 970]